In [19]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import descartes
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import datetime
import numpy as np
import math
import mapclassify
import json
import seaborn
import ipywidgets as widgets
from IPython.display import display
from collections import namedtuple
import time

In [20]:
%matplotlib inline

In [21]:
stations = pd.read_json("http://monitor.pogodynka.pl/api/map/?category=meteo")

In [4]:
stations = stations[['i','n','la','lo']]

Polygons data for voivodeships downloaded from http://www.gugik.gov.pl/pzgik/dane-bez-oplat/dane-z-panstwowego-rejestru-granic-i-powierzchni-jednostek-podzialow-terytorialnych-kraju-prg .

Exact url of the .zip file : ftp://91.223.135.109/prg/jednostki_administracyjne.zip

In [5]:
voi = gpd.read_file('map/voivodeships.shp',encoding='windows-1250')
voi['woj_nazwa'] = ['Śląskie','Opolskie','Wielkopolskie','Zachodniopomorskie','Świętokrzyskie','Kujawsko-Pomorskiej',
                  'Podlaskie','Dolnośląskie','Podkarpackie','Małopolskie','Pomorskie','Warmińsko-Mazurskie', 'Łódzkie','Mazowieckie',
                  'Lubelskie','Lubuskie'] 
voi = voi[['JPT_KOD_JE','geometry','woj_nazwa']]
voi.columns=['voi_id','geometry','voi_name']

In [22]:
voi['geometry'] = voi['geometry'].apply(lambda x: x.simplify(tolerance=0.01)) #for speeding up the map

In [23]:
stations = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.lo, stations.la))

In [24]:
stations['key_temp']=1
voi['key_temp']=1
map_pl = pd.merge(stations,voi,on = 'key_temp',how='outer')

In [25]:
n = map_pl.shape[0]
for x in range(n):
    map_pl.loc[x,'stays'] = map_pl.geometry_y[x].contains(map_pl.geometry_x[x])

In [26]:
map_pl = map_pl[map_pl.stays == True]

In [27]:
map_pl = map_pl.loc[:,['i','n','la','lo','geometry_x','voi_id','voi_name','geometry_y']]
map_pl = map_pl.rename(columns = {'i':'station_code', 'n':'station_name'})

In [28]:
meteo_data = pd.read_pickle('IMGW_data/meteo_data_all.pickle')  

In [29]:
meteo_data = pd.merge(meteo_data, map_pl, on ='station_code',how = 'left' )

In [30]:
#function used to measure performance while creating the map
def stoper(f):
    def new_f(*args,**kwargs):
        start_time = time.time()
        result = f(*args, **kwargs)
        stop_time = time.time()
        print(f'Time of work {stop_time-start_time}')
        return result
    return new_f

In [31]:
avg_temp_voi_dt = pd.DataFrame(meteo_data.groupby(['year','date','voi_id'])['temp_avg'].mean())
avg_temp_voi_dt.reset_index(level=['year','voi_id'],inplace=True)

In [32]:
#generic map per given day
def draw_map(dt):
    one_day_data = avg_temp_voi_dt.loc[dt]
    one_day_data = gpd.GeoDataFrame(pd.merge(one_day_data, voi, how= 'right',left_on= 'voi_id',right_on='voi_id'))
    #layers with and without data
    one_day_data_no = one_day_data[one_day_data.temp_avg.isna()]
    one_day_data_yes = one_day_data[one_day_data.temp_avg.isna() == False]

    #draw
    fig, ax = plt.subplots(1, 1)
    ax.axis('off')

    one_day_data_yes.plot(column='temp_avg',
                cmap='gist_rainbow_r',
                ax=ax,
                legend=True,
                vmin =-25,
                vmax = 30,
                legend_kwds={'label': "Average temperature in a voivodeship"}
               ).set_aspect(1.62) 
    if(one_day_data_no.empty == False):
        one_day_data_no.plot(ax=ax,
                     color='lightgrey').set_aspect(1.62)
    plt.show()

In [33]:
#adding a slider
dates = meteo_data.date.unique()
dates.sort()

slider = widgets.SelectionSlider(
    options=dates,
    value=dates[-1],
    continuous_update=False,
    disabled=False
    )

output_map = widgets.Output()

def react_with_slider(change):
    output_map.clear_output(wait=True)
    with output_map:
        draw_map(change.new)

slider.observe(react_with_slider, names='value')

# function that chooses only the data that applies to dropdown option
def chosen_year(DF):
    dates = np.array(pd.Series(DF.index).unique())
    dates.sort()
    Dummy = namedtuple('Dummy', ['new'])
    dummy = Dummy(dates[-1])
    slider.unobserve(react_with_slider, names='value')
    slider.options =  dates
    slider.value = dates[-1]
    slider.observe(react_with_slider, names='value')
            
    react_with_slider(dummy)

In [34]:
#adding a dropdown
years = avg_temp_voi_dt.year.unique()
years[::-1].sort()
years=years.astype(str)
years = np.insert(years,0,'All')
dropdown = widgets.Dropdown(
    options=list(years),
    value=years[0],
    description='Choose year',
    disabled=False
)

def react_with_dropdown(change):
    output_map.clear_output(wait=True)
    if (change.new == 'All'):
        chosen_year(avg_temp_voi_dt)
    else:
        chosen_year(avg_temp_voi_dt[avg_temp_voi_dt.year == int(change.new)])
dropdown.observe(react_with_dropdown,names='value')
display(dropdown)
display(output_map)
display(slider)
chosen_year(avg_temp_voi_dt)

Dropdown(description='Choose year', options=('All', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2…

Output()

SelectionSlider(continuous_update=False, index=25201, options=(datetime.date(1951, 1, 1), datetime.date(1951, …

#### Please download and execute the notebook to be able to use the map. Below you can find a small preview.

![Map_GIF](screenshots/map_preview.gif)